# Loading the Libraries

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [9]:
train_test_file_path="../evaluation/modeldata-training-and-testing.csv"

In [13]:
all_data = pd.read_csv(train_test_file_path, sep=",", header=0)

In [18]:
all_data.head(5);

In [15]:
def clear_na_data(data):
    # Remove rows with NaN values
    data = data.dropna()
    return data

In [16]:
cleaned_data=clear_na_data(all_data)

In [19]:
cleaned_data.head(5);